In [1]:
%load_ext autoreload
%autoreload 2
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-864c07c4-8eeb-5b23-8d57-eaeb942a9a0f"
from hydra.utils import instantiate
import yaml
import torch
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from PIL import Image
device = "cuda:0"

# Load data and models

In [2]:
os.chdir('/allen/aics/modeling/ritvik/projects/benchmarking_representations/')
save_path = './test_polymorphic_save_embeddings/'

In [3]:
from br.data.get_datamodules import get_data
from cyto_dl.models.utils.mlflow import load_model_from_checkpoint
from br.models.load_models import load_model_from_path
from br.models.save_embeddings import get_pc_loss

def get_data_and_models(dataset_name, batch_size, results_path, debug=False):
    data_list = get_data(dataset_name, batch_size, results_path, debug)
    all_models, run_names, model_sizes = load_model_from_path(dataset_name, results_path) # default list of models in load_models.py
    return data_list, all_models, run_names, model_sizes

In [4]:
dataset_name = 'other_polymorphic'
batch_size = 2
debug=True
results_path = '/allen/aics/modeling/ritvik/projects/benchmarking_representations/configs/results/'
data_list, all_models, run_names, model_sizes = get_data_and_models(dataset_name, batch_size, results_path, debug)

In [6]:
run_names

['equiv_vn',
 'CNN_SO3_pad35_sdf',
 'CNN_SO3_pad35_seg',
 'CNN_sdf_pad35',
 'CNN_seg_pad35']

# Compute embeddings and emissions

In [16]:
from br.models.save_embeddings import save_embeddings

splits_list = ["train", 'val', "test"]
meta_key = None
eval_scaled_img = [True] * 5
eval_scaled_img_params = [
                        {"eval_scaled_img_model_type":"iae",
                          "eval_scaled_img_resolution":32,
                          "gt_mesh_dir":gt_mesh_dir,
                          "gt_scale_factor_dict_path":None,
                          "gt_sampled_pts_dir":gt_sampled_pts_dir,
                          "mesh_ext":"stl"},
                        {"eval_scaled_img_model_type":"sdf",
                          "eval_scaled_img_resolution":32,
                          "gt_mesh_dir":gt_mesh_dir,
                          "gt_scale_factor_dict_path":gt_scale_factor_dict_path,
                          "gt_sampled_pts_dir":None,
                          "mesh_ext":"stl"},
                        {"eval_scaled_img_model_type":"seg",
                          "eval_scaled_img_resolution":32,
                          "gt_mesh_dir":gt_mesh_dir,
                          "gt_scale_factor_dict_path":gt_scale_factor_dict_path,
                          "gt_sampled_pts_dir":None,
                          "mesh_ext":"stl"},
                        {"eval_scaled_img_model_type":"sdf",
                          "eval_scaled_img_resolution":32,
                          "gt_mesh_dir":gt_mesh_dir,
                          "gt_scale_factor_dict_path":gt_scale_factor_dict_path,
                          "gt_sampled_pts_dir":None,
                          "mesh_ext":"stl"},
                        {"eval_scaled_img_model_type":"seg",
                          "eval_scaled_img_resolution":32,
                          "gt_mesh_dir":gt_mesh_dir,
                          "gt_scale_factor_dict_path":gt_scale_factor_dict_path,
                          "gt_sampled_pts_dir":None,
                          "mesh_ext":"stl"},]
loss_eval_list = [torch.nn.MSELoss(reduction='none')]
sample_points_list = [False]*5
skew_scale = None
save_embeddings(
    save_path,
    data_list,
    all_models,
    run_names,
    debug,
    splits_list,
    device,
    meta_key,
    loss_eval_list,
    sample_points_list,
    skew_scale,
    eval_scaled_img,
    eval_scaled_img_params,
)

{'latent_dim': 512, 'spatial_dims': 3, 'in_shape': [1, 35, 35, 35], 'channels': [8, 16, 32, 64, 128, 256, 512], 'strides': [1, 1, 2, 2, 2, 2, 2], 'last_act': None, 'x_label': 'image', 'id_label': None, 'reconstruction_loss': MSELoss(), 'prior': None, 'embedding_prior': 'identity', 'group': 'so3', 'beta': 0, 'act': 'relu', 'norm': 'batch', 'dropout': None, 'bias': True, 'mask_input': True, 'mask_output': True, 'optimizer': functools.partial(<class 'torch.optim.adam.Adam'>, lr=0.001), 'background_value': 2, 'kernel_sizes': [3, 3, 3, 3, 3, 3, 3], 'encoder_padding': [None, None, None, None, None, None, None], 'first_conv_padding_mode': 'replicate', 'num_res_units': 2, 'num_dec_res_units': 0, 'decoder_channels': [512, 256, 128, 64, 32, 16], 'decoder_strides': [2, 2, 2, 2, 1, 1]}